In [57]:
import pandas as pd
import numpy as np
import math

In [58]:
RANDOM_SEED = 13022022

In [59]:
test = pd.read_csv('Task/test.csv')
test.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [60]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1614 entries, 0 to 1613
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Smiles  1614 non-null   object
dtypes: object(1)
memory usage: 12.7+ KB


In [61]:
test.sample(3)

,Smiles
234,Nc1n[nH]c2nnc(-c3c(-c4ccccc4)nn4ccccc34)cc12
1293,O=C(CCc1ccncc1)NNC(=O)N1Cc2ccccc2Oc2ccc(Cl)cc21
91,NS(=O)(=O)c1ccc(N2CCCCS2(=O)=O)cc1


In [62]:
train = pd.read_csv('Task/train.csv')
train.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [63]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5557 entries, 0 to 5556
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Smiles  5557 non-null   object
 1   Active  5557 non-null   bool  
dtypes: bool(1), object(1)
memory usage: 49.0+ KB


In [64]:
train.sample(3)

,Smiles,Active
1531,Cc1nc(-c2ccc(OCC(C)C)c(C#N)c2)sc1C(=O)O,False
2643,C=C(NC(=O)[C@@H](NC(=O)[C@H](Cc1cnc[nH]1)NC(=O...,False
1404,Oc1c(I)cc(Cl)c2cccnc12,False


In [65]:
list(train['Active'].unique())

[False, True]

In [66]:
list_active = {False: 0, True: 1}

In [67]:
train['Active'] = train['Active'].map(list_active)

In [68]:
train.sample()

,Smiles,Active
4221,CCCNC(=O)[C@H]1O[C@@H]1C(=O)N[C@H](C(=O)N1CCC[...,0


In [69]:
train['train'] = 1
test['train'] = 0

In [70]:
df = pd.concat([train, test], ignore_index=True)

In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7171 entries, 0 to 7170
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Smiles  7171 non-null   object 
 1   Active  5557 non-null   float64
 2   train   7171 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 168.2+ KB


In [72]:
df['train'].unique()

array([1, 0], dtype=int64)

In [73]:
df[df['train'] == 1]['Active'].value_counts()

0.00    5351
1.00     206
Name: Active, dtype: int64

In [74]:
from collections import Counter

atoms = Counter()
for i in df['Smiles'].unique():
    for j in i:
        atoms[j] += 1

In [75]:
atoms['C']

67272

In [76]:
new_columnes = list(atoms.keys())

In [77]:
print(new_columnes)

['C', 'O', 'c', '1', '2', '[', 'n', 'H', ']', '(', 'N', ')', '@', '.', 'l', '=', 'S', '3', '4', 'F', '-', 's', '/', '5', 'o', 'a', '+', '#', 'I', 'P', 'B', 'r', '\\', 'Z', '6', '7', '8', 'e', 'A', 'K', 'M', 'g', 'i', 'L', '9']


In [78]:
df['len_smiles'] = df['Smiles'].apply(lambda x: len(x))

In [79]:
df['len_smiles'].max()

707

In [80]:
df['len_smiles'].describe()

count   7171.00
mean      51.12
std       31.88
min        6.00
25%       34.00
50%       46.00
75%       60.00
max      707.00
Name: len_smiles, dtype: float64

In [81]:
df[df['len_smiles'] > 60]['Smiles'].count()

1732

In [82]:
df['smiles_list'] = df['Smiles'].apply(lambda x: list(x))

In [83]:
def count_i(x):
    atoms = Counter()
    for i in x:
        atoms[i] += 1
    return atoms

In [84]:
df['smiles_list'] = df['smiles_list'].apply(lambda x: count_i(x))

In [85]:
for i in new_columnes:
    df[i] = df['smiles_list'].apply(lambda x: x[i])

In [86]:
df.sample(3)

,Smiles,Active,train,len_smiles,smiles_list,C,O,c,1,2,...,7,8,e,A,K,M,g,i,L,9
6759,CC(=O)Nc1nnc(S(N)(=O)=O)s1,nan,0,26,"{'C': 2, '(': 4, '=': 3, 'O': 3, ')': 4, 'N': ...",2,3,2,2,0,...,0,0,0,0,0,0,0,0,0,0
2686,CC(/C=C/C(=O)NO)=C\[C@@H](C)C(=O)c1ccc(N(C)C)cc1,0.00,1,48,"{'C': 11, '(': 6, '/': 2, '=': 4, 'O': 3, ')':...",11,3,6,2,0,...,0,0,0,0,0,0,0,0,0,0
6927,CC(C)c1onc(-c2c(Cl)cccc2Cl)c1COc1ccc(/C=C/c2cc...,nan,0,66,"{'C': 10, '(': 7, ')': 7, 'c': 21, '1': 4, 'o'...",10,3,21,4,4,...,0,0,0,0,0,0,0,0,0,0


In [87]:
df.sample(3)

,Smiles,Active,train,len_smiles,smiles_list,C,O,c,1,2,...,7,8,e,A,K,M,g,i,L,9
3231,CC1(C)S[C@@H]2[C@H](NC(=O)[C@H](N)c3ccccc3)C(=...,0.00,1,71,"{'C': 10, '1': 2, '(': 6, ')': 6, 'S': 1, '[':...",10,4,6,2,2,...,0,0,0,0,0,0,0,0,0,0
7104,NC[C@@H]1O[C@H](O[C@@H]2[C@@H](CO)O[C@@H](O[C@...,nan,0,194,"{'N': 6, 'C': 23, '[': 19, '@': 29, 'H': 19, '...",23,25,0,2,2,...,0,0,0,0,0,0,0,0,0,0
30,Cn1cnc(-c2cc3nccc(Oc4ccc(NC(=S)NC(=O)Cc5ccccc5...,0.00,1,59,"{'C': 4, 'n': 3, '1': 2, 'c': 22, '(': 5, '-':...",4,2,22,2,2,...,0,0,0,0,0,0,0,0,0,0


In [88]:
df.drop(['smiles_list'], axis = 1, inplace = True)

In [89]:
df.set_index(['Smiles'], inplace = True)

In [90]:
df.sample(3)

,Active,train,len_smiles,C,O,c,1,2,[,n,...,7,8,e,A,K,M,g,i,L,9
Smiles,,,,,,,,,,,,,,,,,,,,,
C[C@](O)(COc1ccc(C#N)cc1)C(=O)Nc1ccc(C#N)c(C(F)(F)F)c1,0.00,1,54,7,3,12,4,0,1,0,...,0,0,0,0,0,0,0,0,0,0
CC(C)c1cccc2c1OC(C(C)C)C(=S)N2CC(=O)O,nan,0,37,10,3,6,2,2,0,0,...,0,0,0,0,0,0,0,0,0,0
CC(=O)N[C@H](Cc1c[nH]c2ccccc12)C(=O)OCc1cc(C(F)(F)F)cc(C(F)(F)F)c1,0.00,1,66,8,3,14,4,2,2,1,...,0,0,0,0,0,0,0,0,0,0


In [91]:
train_new = df[df['train'] == 1]
test_new = df[df['train'] == 0]

In [92]:
train_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5557 entries, COc1ccc2[nH]cc(CCN)c2c1 to CCCCNc1ccc(C(=O)OCCN(C)C)cc1.Cl
Data columns (total 48 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Active      5557 non-null   float64
 1   train       5557 non-null   int64  
 2   len_smiles  5557 non-null   int64  
 3   C           5557 non-null   int64  
 4   O           5557 non-null   int64  
 5   c           5557 non-null   int64  
 6   1           5557 non-null   int64  
 7   2           5557 non-null   int64  
 8   [           5557 non-null   int64  
 9   n           5557 non-null   int64  
 10  H           5557 non-null   int64  
 11  ]           5557 non-null   int64  
 12  (           5557 non-null   int64  
 13  N           5557 non-null   int64  
 14  )           5557 non-null   int64  
 15  @           5557 non-null   int64  
 16  .           5557 non-null   int64  
 17  l           5557 non-null   int64  
 18  =           5557 non-nul

In [93]:
X = train_new.drop(['Active'], axis = 1)
y = train_new['Active']

In [94]:
from sklearn import model_selection, datasets, metrics, tree 

In [95]:
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(X, y, 
                                                                                    test_size=0.30, random_state=42)

In [96]:
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

In [97]:
clf = tree.DecisionTreeClassifier()
clf.fit(train_data, train_labels)

DecisionTreeClassifier()

In [98]:
predictions_clf = clf.predict(test_data)
metrics.f1_score(test_labels, predictions_clf)

0.15714285714285714

In [105]:
bagging_trees = BaggingClassifier(clf, n_estimators = 150)
bagging_trees.fit(train_data,train_labels)
predictions_bg = bagging_trees.predict(test_data)

In [106]:
metrics.f1_score(test_labels, predictions_bg)

0.07894736842105261

In [107]:
from lazypredict.Supervised import LazyClassifier

In [108]:
clf_lc = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf_lc.fit(train_data, test_data, train_labels, test_labels)

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:21<00:00,  1.37it/s]


In [109]:
print(models)

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
DecisionTreeClassifier             0.93               0.55     0.55      0.93   
NearestCentroid                    0.75               0.54     0.54      0.83   
ExtraTreeClassifier                0.93               0.53     0.53      0.93   
LabelSpreading                     0.94               0.52     0.52      0.93   
LabelPropagation                   0.94               0.52     0.52      0.93   
XGBClassifier                      0.96               0.52     0.52      0.94   
Perceptron                         0.94               0.51     0.51      0.93   
BaggingClassifier                  0.95               0.51     0.51      0.94   
SGDClassifier                      0.95               0.51     0.51      0.94   
LinearDiscriminantAnalysis         0.95               0.51     0.51      0.94   
QuadraticDiscriminantAnalysi

In [99]:
from sklearn.neighbors import NearestCentroid

In [100]:
NC_model = NearestCentroid()

In [101]:
train_data, test_data, train_labels, test_labels
NC_model.fit(train_data, train_labels)
predictions_clf_tree = NC_model.predict(test_data)
metrics.f1_score(test_labels, predictions_clf_tree)

0.10037174721189593

In [102]:
from sklearn.ensemble import RandomForestClassifier

clf_rf = RandomForestClassifier()


In [103]:
clf_rf.fit(train_data, train_labels)
predictions_clf_rf = clf_rf.predict(test_data)
metrics.f1_score(test_labels, predictions_clf_rf)

0.0273972602739726

In [110]:
test_new_data = test_new.drop(['Active'], axis = 1) 

In [111]:
predictions = clf.predict(test_new_data)

In [112]:
test_new_data['Active'] = predictions

In [113]:
test_new_data.sample(5)

,train,len_smiles,C,O,c,1,2,[,n,H,...,8,e,A,K,M,g,i,L,9,Active
Smiles,,,,,,,,,,,,,,,,,,,,,
CC(C)OC(=O)c1cc2c(OC[C@@H](O)CNC(C)(C)C)cccc2[nH]1,0,50,11,4,8,2,2,2,1,2,...,0,0,0,0,0,0,0,0,0,0.00
CN1CCN(CCCN2c3ccccc3Sc3ccc(C(F)(F)F)cc32)CC1.Cl.Cl,0,50,11,0,12,2,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0.00
CCC/C=C1\OC(=O)C2=C1CCC=C2,0,26,12,2,0,2,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0.00
O=C(O)CC1CC(c2ccccc2)=NO1,0,25,5,3,6,2,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0.00
CCN(CC)CCOC(=O)c1ccc(N)cc1.Cl,0,29,8,2,6,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.00


In [114]:
test_new_data.reset_index(inplace = True)

In [115]:
test_new_data.sample(5)

,Smiles,train,len_smiles,C,O,c,1,2,[,n,...,8,e,A,K,M,g,i,L,9,Active
1051,CCc1cc(/C(C)=N/OCc2ccc(C3CCCCC3)c(C(F)(F)F)c2)...,0,65,17,3,12,4,2,0,0,...,0,0,0,0,0,0,0,0,0,0.00
1430,CCCN(CCC)CCc1ccc(OC)c(OCCc2ccccc2)c1.Cl,0,39,12,2,12,2,2,0,0,...,0,0,0,0,0,0,0,0,0,0.00
577,CN(C)CCCN1c2ccccc2Sc2ccc(C(F)(F)F)cc21.Cl,0,41,7,0,12,2,4,0,0,...,0,0,0,0,0,0,0,0,0,0.00
839,CCCn1c(=O)c2nc(-c3ccccc3)[nH]c2n(CCC)c1=O,0,41,6,2,11,2,2,1,4,...,0,0,0,0,0,0,0,0,0,0.00
1314,CCCC[C@H]1OC(=O)c2ccccc21,0,25,6,2,6,2,2,1,0,...,0,0,0,0,0,0,0,0,0,0.00


In [116]:
submission = test_new_data[['Smiles', 'Active']]

In [117]:
submission.sample(4)

,Smiles,Active
280,CNC(=O)c1cc2c(cn1)[nH]c1ccccc12,0.00
919,Cn1c(SCC(=O)N2CCC[C@H]2C(=O)Nc2ccccc2-c2ccccc2...,0.00
910,O=C(Nc1ccc(CN2CCOCC2)cc1)c1ccc(-c2cccc(Cl)c2)o1,0.00
1310,O=C(Nc1ccccc1)Nc1ccc(-c2ccc(C(=O)[C@@H]3CCC[C@...,0.00


In [118]:
submission['Active'] = submission['Active'].apply(lambda x: int(x))

In [119]:
submission.to_csv('submission.csv', index=False)